# Homework 3.1 - Introduction to clustering with scikit-learn

### Introduction to the notebook

This first notebook will walk through the basics of using skikit-learn for building a classification model using k-means clustering. K means clustering is an unsupervised machine learning technique that groups data points based on their similarity or closeness of features. It works by finding the centre (mean) of each cluster and assigning data points to the nearest cluster. 

Thanks to the scikit-learn library, we can leverage ready-made functions that can accelerate the process of building a model. This notebook will explore this process using a sample dataset...

In [ ]:
# Install scikit-learn
%pip install scikit-learn

In [ ]:
# Import libraries

import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler # for standardising the data
from sklearn.datasets import load_wine # For loading the dataset
from sklearn.cluster import KMeans # For kmeans clustering
from sklearn.decomposition import PCA # For PCA decomposition - dimensionality reduction

In [ ]:
# Let's load the data and take a peek...
raw_data = load_wine(as_frame=True)
wine_df = raw_data.frame

wine_df.head()

In [ ]:
targets = wine_df['target']
wine_df.drop('target', axis=1, inplace=True) # As clustering is unsupervised, we don't need the target column.
wine_df.columns

In [ ]:
# Feel free to include your own code to explore the data with plots and summary statistics as done in week 2's homework...

In [ ]:
scaler = StandardScaler() # Create an instance of the scaler, this will be used to scale the data before clustering because the features are on different scales so we need to standardise them
features_ = scaler.fit(wine_df) # Fit the scaler to the training data
features = features_.transform(wine_df) # Transform the features using the scaler
scaled_df = pd.DataFrame(features, columns=wine_df.columns) # Create a dataframe from the scaled features
X = scaled_df.values # Get the values from the dataframe

scaled_df.head()

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Ignore the FutureWarning that is thrown when plotting the elbow plot

wcss = [] # Create an empty list to store the within cluster sum of squares (WCSS) for each iteration of the kmeans algorithm. The WCSS is the sum of the squared distance between each point and the centroid in a cluster. The lower the WCSS the better the clustering.
for i in range(1, 16): # Iterate through the kmeans algorithm 15 times, each time increasing the number of clusters by 1.
    kmeans_model = KMeans(n_clusters=i, random_state=42) # Create an instance of the kmeans model, setting the number of clusters, the initialisation method and the random state.
    kmeans_model.fit(X) # Fit the model to the data.
    wcss.append(kmeans_model.inertia_)

wcss

### Determine the optimal number of clusters using the 'elbow' method

The elbow method is a heuristic used in determining the number of clusters in a specific dataset. The method consists of plotting the explained variation as a function of the number of clusters and picking the elbow of the curve as the number of clusters to use. Naturally, increasing the number of clusters will improve the fit of the model, since there are more clusters to use, but that at some point this is over-fitting, and the 'elbow' reflects this. 

In [ ]:
plt.plot([x for x in range(1, 16)], wcss, 'gs-') # Plot the elbow plot, the x axis is the number of clusters and the y axis is the WCSS.
plt.xlabel("Values of 'k'")
plt.ylabel('WCSS')
plt.plot([3], [wcss[2]], 'ro', ms=12, mfc='r') # Circle the elbow point on the plot, this is the optimal number of clusters.
plt.show() # Elbow method to determine the optimal number of clusters. The elbow method is a heuristic method of interpretation and validation of consistency within cluster analysis designed to help find the appropriate number of clusters in a dataset. The optimal number of clusters is the value of k at the elbow i.e. the point after which the WCSS does not decrease significantly with every iteration.

In [ ]:
kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42) # Create an instance of the kmeans algorithm with 3 clusters because the elbow method suggests that 3 is the optimal number of clusters
kmeans.fit(X)
centres = kmeans.cluster_centers_ # Get the coordinates of the centroids for each cluster. As there are 15 features, there are 15 dimensions. We can't visualise 15 dimensions so we need to reduce the dimensions to 2 using PCA

centres

### Using organic features from the original dataset

Usually we would do some exploratory data analysis to determine which features we will use, but in this case we will use two randomly chosen features...

In [ ]:
# Let's try using 2 random features from the original dataset to see if we can improve the clustering. In reality, we would use domain knowledge to select the features to use for clustering - we would do some exploratory data analysis to see which features are most correlated with the target variable and select those features for clustering.

X_ = scaled_df[['alcohol', 'flavanoids']].values # Get the features from the dataframe

# We need the index of the features we selected to get the coordinates of the centroids for each cluster. We can get the index of the features from the dataframe.
alcohol_index = 0
flavanoids_index = 6

alcohol_centroids = centres[:, alcohol_index] # Get the coordinates of the centroids for each cluster for the alcohol feature
flavanoids_centroids = centres[:, flavanoids_index] # Get the coordinates of the centroids for each cluster for the flavanoids feature

#### Assessing the model

K-means clustering does not use any labels or class information to assign the clusters. Therefore, the labels that k-means assigns to the clusters are arbitrary and may not match the true labels of the data points. Plotting the clusters next to the original data is a useful way to visualise the results of k-means clustering, especially if the data has two or three dimensions.

The silhouette score is a metric that evaluates how well each data point fits in its assigned cluster, and how well it is separated from other clusters. The silhouette score ranges from -1 to 1, where a higher value means a better clustering result. A silhouette score of 1 means that the data point is very similar to its own cluster, and very different from other clusters. A silhouette score of 0 means that the data point is close to the boundary between two clusters. A silhouette score of -1 means that the data point is more similar to another cluster than to its own cluster. In Python, we can use the `sklearn` library to perform k-means clustering and calculate the silhouette score. The `sklearn.metrics` module provides two functions: `silhouette_score` and `silhouette_samples`. The `silhouette_score` function returns the average silhouette score for the data set, given the cluster labels. The `silhouette_samples` function returns the silhouette score for each data point, which can be used to plot a silhouette diagram.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5)) # Create a figure with two subplots.

ax1.scatter(X_[:,0], X_[:,1], c=kmeans.labels_) # Plot the clusters and the centroids
ax1.scatter(alcohol_centroids[:], flavanoids_centroids[:], marker='x', s=100,c='red')
ax1.set_xlabel('alcohol')
ax1.set_ylabel('flavanoids')
ax1.set_title('Predicted clusters')
ax2.scatter(X_[:,0], X_[:,1], c=targets) # Plot the original data with the target variable.
ax2.set_xlabel('alcohol')
ax2.set_ylabel('flavanoids')
ax2.set_title('Original data')
plt.tight_layout()
plt.show()

In [ ]:
# We can use the silhouette score to evaluate the quality of the clusters. The silhouette score is a measure of how similar an object is to its own cluster compared to other clusters. The silhouette score ranges from -1 to 1. A score of 1 indicates that the object is far away from the neighbouring clusters. A score of 0 indicates that the object is on or very close to the decision boundary between two neighbouring clusters and negative values indicate that those samples might have been assigned to the wrong cluster. The higher the silhouette score the better the clustering.
from sklearn.metrics import silhouette_score

print(f'Silhouette score: {silhouette_score(X_, kmeans.labels_)}') # Get the silhouette score for the clusters

### Principal component analysis (optional)

As our chosen dataset has a large number of features (dimensions), we could choose to break it down using principal component analysis. Principal component analysis, or PCA, is a method that reduces the dimensionality of a data set by transforming a large set of variables into a smaller one that still contains most of the information in the large set. This makes it easier to explore and visualise the data, as well as to perform machine learning tasks on it.

The basic idea of PCA is to find the directions in which the data varies the most, and project the data onto those directions. These directions are called principal components, and they are orthogonal to each other. The first principal component captures the most variance in the data, the second one captures the second most, and so on. By choosing a subset of principal components, we can reduce the number of dimensions while preserving as much information as possible.

_Further reading on this can be found in the `README.md` file. Please note this is an extension task._

In [ ]:
# We can find the optimal number of components for PCA. We can do this by plotting the cumulative explained variance ratio against the number of components. We aim for the cumulative explained variance ratio to be as close to 1 as possible. The cumulative explained variance ratio is the proportion of the variance in the original data that is explained by the components. The more components we use the more variance we can explain but we want to use as few components as possible to avoid overfitting. We can see from the plot that 2 components explains around 60% of the variance in the original data so we will use 2 components for PCA - this means we can visualise the clusters in 2 dimensions.
pca = PCA() # Create an instance of the PCA model
pca.fit(X) # Fit the model to the data
cumulative_explained_variance = [] # Create an empty list to store the cumulative explained variance for each component
for i in range(1, 14): # Iterate through the number of components
    cumulative_explained_variance.append(sum(pca.explained_variance_ratio_[:i])) # Append the cumulative explained variance for each component to the list
plt.plot([x for x in range(1, 14)], cumulative_explained_variance, 'gs-') # Plot the cumulative explained variance against the number of components
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.plot([2], [cumulative_explained_variance[1]], 'ro', ms=12, mfc='r') # We will use 2 components for PCA as, although it doesn't have the highest cumulative explained variance, it means we can visualise the clusters in 2 dimensions.
plt.show()

In [ ]:
pca = PCA(n_components=2) # Create an instance of the PCA model with 2 components
reduced_X = pd.DataFrame(data=pca.fit_transform(X.copy()), columns=['PCA-feature1', 'PCA-feature2']) # Create a dataframe from the reduced features.
centres_pca = pca.transform(centres) # Transform the coordinates of the centroids for each cluster using the PCA algorithm.

centres_pca

In [ ]:
# Let's plot the clusters next to the original data in two different subplots. As clustering is unsupervised, we don't have a target variable to compare the clusters to. However, we can compare the clusters to the original data to see if the clusters make sense.
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5)) # Create a figure with two subplots.
fig.suptitle('Wine Clusters')
ax1.scatter(reduced_X['PCA-feature1'], reduced_X['PCA-feature2'], c=kmeans.labels_) # Plot the clusters and the centroids after reducing the dimensionality of the data to 2 dimensions using the PCA algorithm
ax1.scatter(centres_pca[:,0], centres_pca[:,1], marker='x', s=100,c='red')
ax1.set_xlabel('PCA-feature1')
ax1.set_ylabel('PCA-feature2')
ax1.set_title('Predicted clusters')
ax2.scatter(reduced_X['PCA-feature1'], reduced_X['PCA-feature2'], c=targets) # Plot the original data with the target variable.
ax2.set_xlabel('PCA-feature1')
ax2.set_ylabel('PCA-feature2')
ax2.set_title('Original data')
plt.tight_layout()
plt.show()

In [ ]:
print(f'Silhouette score: {silhouette_score(reduced_X, kmeans.labels_)}') # Print the silhouette score for the clusters.